In [ ]:
import pandas as pd
import numpy as np
import gzip
import sys
sys.path.append('../Util')
import PreProcessingText as ppt
import BERTopicUtils as btu
import TextClustering as tc
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from bertopic.representation import MaximalMarginalRelevance, KeyBERTInspired, PartOfSpeech
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
import hdbscan
from bertopic.vectorizers import ClassTfidfTransformer
from transformers import pipeline

In [2]:
PATH = '../Analyze_files/CombiningAnalysisCompleteDataset'

In [40]:
bt_content = BERTopic.load(f"{PATH}/ContentAnalysis/ModelsContent/topic_model_all-MiniLM-L6-v2_190_20n_8dim", embedding_model='all-MiniLM-L6-v2')
bt_thread = BERTopic.load(f"{PATH}/ThreadAnalysis/Models/topic_model_all-MiniLM-L6-v2_150_20n", embedding_model='all-MiniLM-L6-v2')

In [45]:
merged_model_thread_content = BERTopic.merge_models([bt_thread, bt_content], min_similarity=0.8)
merged_model_content_thread = BERTopic.merge_models([bt_content, bt_thread], min_similarity=0.8)

In [46]:
print(len(merged_model_content_thread.get_topic_info()))
print(len(merged_model_thread_content.get_topic_info()))

133
120


In [47]:
merged_model_content_thread.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,110608,-1_vendor_order_nt_market,"[vendor, order, nt, market, link, get, like, t...",NaN
1,0,19104,0_cart_weed_strain_thc,"[cart, weed, strain, thc, bud, price, product,...",NaN
2,1,6011,1_deposit_address_ticket_btc,"[deposit, address, ticket, btc, wallet, deposi...",NaN
3,2,6350,2_key_pgp_account_pgp key,"[key, pgp, account, pgp key, password, message...",NaN
4,3,3368,3_order_shipped_ordered_day,"[order, shipped, ordered, day, week, marked, p...",NaN
...,...,...,...,...,...
128,127,397,40_withdraw_withdrawal_withdrawl_withdraws,"[withdraw, withdrawal, withdrawl, withdraws, b...",NaN
129,128,281,48_heard_happened_anyone_anyone heard,"[heard, happened, anyone, anyone heard, thewiz...",NaN
130,129,308,49_benzos_benzo_rc_benzo vendor,"[benzos, benzo, rc, benzo vendor, rc benzos, b...",NaN
131,130,326,50_fraud_fraudsters_fraud vendor_loan fraud,"[fraud, fraudsters, fraud vendor, loan fraud, ...",NaN


In [48]:
bt_content.get_topic_info().tail(5)

,Topic,Count,Name,CustomName,Representation,Representative_Docs
117,116,1754,116_water_acetone_powder_dry,water - powder - acetone,"[water, acetone, powder, dry, ml, solution, di...",[acetone wash basic bare minimum lazy simple w...
118,117,218,117_witchman_link witchman_link_witchman link,witchman - witchman link,"[witchman, link witchman, link, witchman link,...","[try witchman, link rtuna rtuna link link witc..."
119,118,210,118_tochka_market_tochka market_tochka tochka,tochka market - market - tochka,"[tochka, market, tochka market, tochka tochka,...","[tochka really, tochka good, tochka]"
120,119,2589,119_post_know guy know_guy know guy_know guy,know guy know - post - guy know guy,"[post, know guy know, guy know guy, know guy, ...",[totally read robin williams voice great guy s...
121,120,509,120_subdread_sub_post_subdreads,subdread - sub - post,"[subdread, sub, post, subdreads, create, dread...","[found subdread, neither subdread, tried creat..."


In [49]:
merged_model_content_thread.get_topic_info().tail(15)

,Topic,Count,Name,Representation,Representative_Docs
118,117,218,117_witchman_link witchman_link_witchman link,"[witchman, link witchman, link, witchman link,...",NaN
119,118,210,118_tochka_market_tochka market_tochka tochka,"[tochka, market, tochka market, tochka tochka,...",NaN
120,119,2589,119_post_know guy know_guy know guy_know guy,"[post, know guy know, guy know guy, know guy, ...",NaN
121,120,509,120_subdread_sub_post_subdreads,"[subdread, sub, post, subdreads, create, dread...",NaN
122,121,999,10_crosspost_giveaway_review crosspost_crosspo...,"[crosspost, giveaway, review crosspost, crossp...",NaN
123,122,608,21_oxycodone_mg_oxy_opiate,"[oxycodone, mg, oxy, opiate, opiateconnect, ox...",NaN
124,123,556,23_drug_drugsuk_drugs_selling drug,"[drug, drugsuk, drugs, selling drug, drug deal...",NaN
125,124,471,31_hacked_hacker_hacking_job,"[hacked, hacker, hacking, job, lfw, malware, h...",NaN
126,125,470,32_login_account_password_log,"[login, account, password, log, fa, error, reg...",NaN
127,126,382,38_cc_cvv_vbv_cc vendor,"[cc, cvv, vbv, cc vendor, cc cvv, non, non vbv...",NaN


In [50]:
merged_model_content_thread.visualize_hierarchy(custom_labels=True)

In [51]:
merged_model_content_thread.visualize_topics()

In [52]:
merged_model_content_thread.visualize_barchart(top_n_topics=250, custom_labels=True, n_words=10)

In [57]:
bt_thread.get_topic_info().tail(5)

,Topic,Count,Name,CustomName,Representation,Representative_Docs
64,63,202,63_dmt_dmt vendor_dmt vape_odsmt,dmt vape - dmt - dmt vendor,"[dmt, dmt vendor, dmt vape, odsmt, dmt dmt, ch...","[best dmt vendor, dmt, vendor dmt]"
65,64,187,64_captcha_rapture_rapture market_captcha captcha,capcha,"[captcha, rapture, rapture market, captcha cap...","[captcha, use rapture registration login captc..."
66,65,198,65_chemical_research_research chemical_chems,research chemical - research - chemical,"[chemical, research, research chemical, chems,...",[chem theory honorable research chemical suppl...
67,66,222,66_tor_tor browser_browser_tor network,tor browser,"[tor, tor browser, browser, tor network, netwo...","[tor browser help, review tor browser, tor bro..."
68,67,178,67_mephedrone_meopcp_mxe_mescaline,methamphetamine - pills,"[mephedrone, meopcp, mxe, mescaline, mmc, meph...",[eurovalz new stock list mxe mephedrone availa...


In [59]:
merged_model_thread_content.get_topic_info().tail(55)

,Topic,Count,Name,Representation,Representative_Docs
65,64,713,64_captcha_rapture_rapture market_captcha captcha,"[captcha, rapture, rapture market, captcha cap...",NaN
66,65,1262,65_chemical_research_research chemical_chems,"[chemical, research, research chemical, chems,...",NaN
67,66,2385,66_tor_tor browser_browser_tor network,"[tor, tor browser, browser, tor network, netwo...",NaN
68,67,178,67_mephedrone_meopcp_mxe_mescaline,"[mephedrone, meopcp, mxe, mescaline, mmc, meph...",NaN
69,68,4584,5_thanks_thank_lol_man,"[thanks, thank, lol, man, bro, good, nice, wor...",NaN
70,69,3425,6_ship_country_eu_shipping,"[ship, country, eu, shipping, uk, usa, germany...",NaN
71,70,2214,14_drug_police_get_nt,"[drug, police, get, nt, house, cop, people, la...",NaN
72,71,2829,26_box_mail_package_address,"[box, mail, package, address, po, po box, lett...",NaN
73,72,1332,37_link_link link_point link comment_link poin...,"[link, link link, point link comment, link poi...",NaN
74,73,1499,38_pack_week_day_ordered,"[pack, week, day, ordered, land, got pack, got...",NaN


In [55]:
merged_model_thread_content.visualize_hierarchy(custom_labels=True)

In [53]:
merged_model_thread_content.visualize_topics()

In [54]:
merged_model_thread_content.visualize_barchart(top_n_topics=250, custom_labels=True, n_words=10)